# Computer-Aided Vaccine Design

In this practical we will implement a neoeptiope selection framework  based on Toussaint et al methematical model.

This will include:

1) Neopeptide construction from somatic mutations
2) Neoeptioep prediction
3) Neoepitope selection under constraints
4) Vaccine assembly

We will cover all three steps in this tutorial. 

## Prerequisites

Please clone the repository and install the following packages environment 
- PySAM (Python API to read and wirte SAM/BAM/VCF/BCF)
- Pyomo (Python API for Integer linear programming)
- Epytope (A Python Framework for Immunoinformatics)
- GLPK (An ILP solver

or use the pre-existing conda:
```
pip uninstall pyomo
pip install pyomo
```

You can install the packages with the following commands:

**GLPK**:
!apt-get install glpk-utils

**Epytope**:
pip install git+https://github.com/KohlbacherLab/epytope



Than please uninstall pyomo and reinstall it again:


**Pyomo**
```
pip uninstall pyomo
pip install pyomo
```


## 1. Neoepitope construction

We will start we implementing functions to read VEP files (output format of the ENSEMBLES Variant effect Predictor, which we used to annotate identified somatic mutations using MuTec2, and create alterated peptides form the variances.
You can finde the exmaple file under ```./data/VEP_example.vep```.

To this end, we need:

1) Collect nonsynonymous single point variances for a transcript 
2) Extract the reference transcript sequence 
3) Integrate the somatic mutations
4) Tranlsate the RNA sequence into its amino acid sequence
5) Create all possible k-mers (while keeping track to which protein it belongt); k in [9, 11]

Task 1.1: Implement a VEP reader and a representations of ```Variants```
----------


        

Task 1.2: Implement a neopeptide generator function that consumes a list of ```Variants``` an generates a list of ```Peptides```of length k
----------
Assume that the variants were created usign the human reference genome GRCh38

## 2: Neoepitope prediction

Once we have created all possible possible alterated neopeptides we need to predict which can bind to HLA and subsequently will be presented on the surface of the tumor.

To this end, we will use MHCflurry. MHCflurry 2.0 is a Deep Learning model using feed-forward neural networks with skip-connections. If you want to learn more about the model check out their paper [here](https://www.cell.com/cell-systems/fulltext/S2405-4712(20)30239-8?_returnURL=https%3A%2F%2Flinkinghub.elsevier.com%2Fretrieve%2Fpii%2FS2405471220302398%3Fshowall%3Dtrue).

You cand find the github repository [here](https://github.com/openvax/mhcflurry) and an example of its API [here](https://github.com/openvax/mhcflurry/blob/master/notebooks/example1.ipynb).

![img](./img/mhcflurry.jpg)

Task 2.1: Implement a Neoepitope function who consums a list of generated ```Peptides``` and predicted their binding affinity to a list of ```HLAs```
-----
Assume the patients HLA-I genotype is HLA-A\*02:01, HLA-B\*07:02, and HLA-C\*02:02

## 3. Epitope Selection

To select the optimal set of epitopes for vaccination we will rebuild OptiTope, Toussaints et al.'s epitope selection ILP, using Pyomo. Pyomo ist a Python-based modeling language for constraint optimization problems. Besides implementing the ILP model, your job will be to come up with a Objective function that is specific for neoeptiope selection and the document & justification your formulation.

### 3.1 Overview

ILP formulations have five essential element: `Sets`, `Parameters`, `Variables`, `Objectives`, and `Constraints`. The letter two use sets as indices for parameters and variables to formulate the equations that make up the objectives and constraints.


As a reminder, the complete formulation of OptiTope looks like this:


\begin{align}
   \text{Sets:}&&&\\
   \\
    E&&&\text{Set of epitopes}\\
    H&&&\text{Set of HLA alleles}\\
    I(h)\subseteq E&~\forall~h \in H&&\text{Set of epitopes binding to HLA $h$}\\
    \\
    \text{Parameters:}&&&\\
    \\
    i_{e, h}&&&\text{Immunogenicity of epitope $e$ bound to HLA $h$}\\
    p_h&&&\text{Allele frequency of HLA $h$ in target population}\\
    k&&&\text{Upper limit of selected epitopes}\\
    \tau_{H}&&&\text{Minimum of covered HLA alleles}\\
    \\
    \text{Variables:}&&&\\
    \\
    x_e \in \{0,1\}&~\forall~e \in E&&\text{Dicision variable whether epitope $e$ is selected}\\
    y_h\in \{0,1\}&~\forall~h \in H&&\text{Dicision variable whether HLA $h$ is covered by the selected epitopes}\\
    \\
    \text{Objective:}&&&\\
    \\
    \arg\max_{\mathbf{x}} &\sum_{h \in H} p_h \sum_{e \in E} x_ei_{e,h}&&\\
    \\
    \text{Constraints:}&&&\\
    \\
    \sum_{e\in E} x_e \leq k&&&\\
    \sum_{e\in I(h)} x_e \geq y_h &~\forall h \in H&&\\
    \sum_{h \in H} y_h \leq \tau_{H} &~\forall h \in H&&\\
\end{align}




### 3.2 Abstract Versus Concrete Models

*(Adopted from Pyomo Documentation)*

A mathematical model can be defined using symbols that represent data
values.  For example, the following equations represent a linear program
(LP) to find optimal values for the vector $x$ with parameters
$n$ and $b$, and parameter vectors $a$ and $c$:


\begin{array}{lll}
    \min       & \sum_{j=1}^n c_j x_j &\\
     \mathrm{s.t.} & \sum_{j=1}^n a_{ij} x_j \geq b_i & \forall i = 1 \ldots m\\
               & x_j \geq 0 & \forall j = 1 \ldots n
\end{array}

<div class="alert alert-block alert-info">
   As a convenience, we use the symbol $\forall$ to mean "for all"
   or "for each."
</div>

We call this an *abstract* or *symbolic* mathematical model since it
relies on unspecified parameter values.  Data values can be used to
specify a *model instance*.  The ``AbstractModel`` class provides a
context for defining and initializing abstract optimization models in
Pyomo when the data values will be supplied at the time a solution is to
be obtained.

In many contexts, a mathematical model can and should be directly
defined with the data values supplied at the time of the model
definition.  We call these *concrete* mathematical models.  For example,
the following LP model is a concrete instance of the previous abstract
model:


\begin{array}{ll}
    \min       & 2 x_1 + 3 x_2\\
     \mathrm{s.t.} & 3 x_1 + 4 x_2 \geq 1\\
               & x_1, x_2 \geq 0
\end{array}

The ``ConcreteModel`` class is used to define concrete optimization
models in Pyomo.

<div class="alert alert-block alert-info">
   Python programmers will probably prefer to write concrete models,
   while users of some other algebraic modeling languages may tend to
   prefer to write abstract models.  The choice is largely a matter of
   taste; some applications may be a little more straightforward using
   one or the other.
</div>


We will work with a ``ConcreteModel``. You can find an example of a ``ConcreteModel`` [here](https://pyomo.readthedocs.io/en/latest/pyomo_overview/simple_examples.html#a-simple-concrete-pyomo-model).

Task 3.1: ConcreteModel initialization
----------
1) Import the necessary Pyomo modules and initialize a concrete model.



### 3.3 Set and Sets of Sets
*(Adopted from Pyomo Documentation)*


Sets can be declared using the `Set` and `RangeSet` functions or by
assigning set expressions.  The simplest set declaration creates a set
and postpones creation of its members:

`
model.A = Set()
`

The ``Set`` function takes optional arguments such as:

- doc = String describing the set
- dimen = Dimension of the members of the set
- filter = A boolean function used during construction to indicate if a
  potential new member should be assigned to the set
- initialize = A function that returns the members to initialize the set.
- ordered = A boolean indicator that the set is ordered; the default is `False`
- validate = A boolean function that validates new member data
- virtual = A boolean indicator that the set will never have elements;
  it is unusual for a modeler to create a virtual set; they are
  typically used as domains for sets, parameters and variables
- within = Set used for validation; it is a super-set of the set being declared.


The `initialize` option can refer to a Python set, which can be
returned by a function or given directly as in

`
model.D = Set(initialize=['red', 'green', 'blue'])
`

If sets are given as arguments to `Set` without keywords, they are
interpreted as indexes for an array of sets. For example, to create an
array of sets that is indexed by the members of the set `model.A`, use

`
model.E = Set(model.A)
`

This is not restricted to one-dimentional indices. You can specify abitrary dimentional index structures.

You can use `initialize` to construct a concrete array of sets either by providing a function that takes the model and an element of `model.A`as input, and returns the entire set, or by providing a dictionary where the `key` is the an element of `model.A` and the `value` the entire set.  


You can finde more detail [here](https://pyomo.readthedocs.io/en/latest/pyomo_overview/simple_examples.html#a-simple-concrete-pyomo-model)


Task 3.2: OptiTope's Set definition
---
Define and initialize the Sets of OptiTope with the predicted epitope data.


### 2.4 Parameters 
*(Adopted from Poymo Documentation)*

The word “parameters” is used in many settings. When discussing a Pyomo model, we use the word to refer to data that must be provided in order to find an optimal (or good) assignment of values to the decision variables. Parameters are declared with the `Param` function, which takes arguments that are somewhat similar to the Set function. For example, the following code snippet declares sets `model.A`, `model.B` and then a parameter array `model.P` that is indexed by `model.A`:

```
model.A = Set()
model.B = Set()
model.P = Param(model.A, model.B)
```

In addition to sets that serve as indexes, the `Param` function takes the following command options:

- default = The value absent any other specification.
- doc = String describing the parameter
- initialize = A function (or Python object) that returns the members to initialize the parameter values.
- validate = A boolean function with arguments that are the prospective parameter value, the parameter indices and the model.
- within = Set used for validation; it specifies the domain of the parameter values.

These options perform in the same way as they do for Set. For example, suppose that `model.A = RangeSet(1,3)`, then there are many ways to create a parameter that is a square matrix with 9, 16, 25 on the main diagonal zeros elsewhere, here are two ways to do it. First using a Python object to initialize:

```
v={}
v[1,1] = 9
v[2,2] = 16
v[3,3] = 25
model.S = Param(model.A, model.A, initialize=v, default=0)
```

And now using an initialization function that is automatically called once for each index tuple (remember that we are assuming that `model.A` contains 1,2,3)

```
def s_init(model, i, j):
    if i == j:
        return i*i
    else:
        return 0.0
model.S1 = Param(model.A, model.A, initialize=s_init)
```

In this example, the index set contained integers, but index sets need not be numeric. It is very common to use strings.

Task 3.3: OptiTope's Parameter definition
---

Define and initialize the Paremters of OptiTope. Since the predicted binding affinities are IC50 values, we have to transform the data with the following functio:

\begin{equation}
i_{e,h} = min(1, max(0, 1 - \log_{50k}(\text{IC50}_{e,h})))
\end{equation}

### 3.5 Variables
*(Adopted from Pyomo Documentation)*

Variables are intended to ultimately be given values by an optimization package. They are declared and optionally bounded, given initial values, and documented using the Pyomo `Var` function. If index sets are given as arguments to this function they are used to index the variable. Other optional directives include:

- bounds = A function (or Python object) that gives a (lower,upper) bound pair for the variable
- domain = A set that is a super-set of the values the variable can take on.
- initialize = A function (or Python object) that gives a starting value for the variable; this is particularly important for non-linear models
- within = (synonym for domain)

The following code snippet illustrates some aspects of these options by declaring a singleton (i.e. unindexed) variable named `model.LumberJack` that will take on real values between zero and 6 and it initialized to be 1.5:

`
model.LumberJack = Var(within=NonNegativeReals, bounds=(0,6), initialize=1.5)
`


For more details see [here](https://pyomo.readthedocs.io/en/latest/pyomo_modeling_components/Variables.html).


Task 3.4: OptiTope's Variable definition
---
Define OptiTope's variables


### 3.6 Objectives
*(Adopted from Pyomos Documentation)*

An objective is a function of variables that returns a value that an optimization package attempts to maximize or minimize. The `Objective` function in Pyomo declares an objective. Although other mechanisms are possible, this function is typically passed the name of another function that gives the expression. Here is a very simple version of such a function that assumes `model.x` has previously been declared as a `Var`:


    def ObjRule(model):
        return 2*model.x[1] + 3*model.x[2]
    model.g = Objective(rule=ObjRule)


It is more common for an objective function to refer to parameters as in this example that assumes that `model.p` has been declared as a `Param` and that `model.x` has been declared with the same index set, while `model.y` has been declared as a singleton:


    def profrul(model):
        return summation(model.p, model.x) + model.y
    model.Obj = Objective(rule=ObjRule, sense=maximize)


This example uses the `sense` option to specify maximization. The default `sense` is minimize.

Task 3.5: OptiTope's Objective definition
---
Define OptiTope's objective function


### 3.7 Constraints
*(Adopted from Pyomos Documentation)*

Most constraints are specified using equality or inequality expressions that are created using a rule, which is a Python function. For example, if the variable `model.x` has the indexes ‘butter’ and ‘scones’, then this constraint limits the sum over these indexes to be exactly three:

    def teaOKrule(model):
        return(model.x['butter'] + model.x['scones'] == 3)
    model.TeaConst = Constraint(rule=teaOKrule)

Instead of expressions involving equality (==) or inequalities (<= or >=), constraints can also be expressed using a 3-tuple if the form (lb, expr, ub) where lb and ub can be None, which is interpreted as lb <= expr <= ub. Variables can appear only in the middle expr. For example, the following two constraint declarations have the same meaning:

    model.x = Var()

    def aRule(model):
       return model.x >= 2
    model.Boundx = Constraint(rule=aRule)

    def bRule(model):
       return (2, model.x, None)
    model.boundx = Constraint(rule=bRule)

Constraints (and objectives) can be indexed by lists or sets. When the declaration contains lists or sets as arguments, the elements are iteratively passed to the rule function. If there is more than one, then the cross product is sent. For example the following constraint could be interpreted as placing a budget of `i` on the ith item to buy where the cost per item is given by the parameter `model.a`:

    model.A = RangeSet(1,10)
    model.a = Param(model.A, within=PositiveReals)
    model.ToBuy = Var(model.A)
    def bud_rule(model, i):
        return model.a[i]*model.ToBuy[i] <= i
    aBudget = Constraint(model.A, rule=bud_rule)

Task 3.6: OptiTope's Constraints
---
Define OptiTope's constraints

### 3.8 Solving a `ConcreteModels`

If you have a ConcreteModel, add these lines at the bottom of your Python script to solve it

    opt = pyo.SolverFactory('glpk')
    res = opt.solve(model) 
    
    
To load the optimal parameters into your model you can use this snippet:

    model.solutions.load_from(res)
    selected_peptides = [x for x in self.instance.x if 0.9 <= model.x[x].value <= 1.2]


Task 3.7: Define now your own objective function for neoepitope selection
---
Please document and justify your choices as well. You may add additional constraints if naccessary.

**NOTE**: You need to turn of the OptiTope's original objective function to solve the problem with your newly defined objective function. You can do so with ```model.<name_of_objective_function>.deactivate()```


## 4. Epitope Assembly

Once we have selected the top k neoepitopes for vaccination we need to assembly those to the final vaccine construct.
Epitope assembly is concerned with finding the best arregement of epitopes to recover all selected peptides after proteasomal cleavage.

This problem can be conveniently formulated as [Traveling Salesperson Problem (TSP)](https://en.wikipedia.org/wiki/Travelling_salesman_problem). 

The TSP tries to find the shortest round trip that visists each city at most once!

To translate the assembly problem into a TSP, we interprete the selected epitopes as cities and the negative cleavage likelihood of two adjecent epitopes as the distance between the citities:

![img](./img/TSP.png)


There are many formulations of the TSP, differing in the number of constraints and in the tightness of the optimality gap of the LP relaxation. A particularly often used formulation was proposed by Miller, Tucker, and Zemlin:

\begin{align}
&\max_{x}\sum_{a,b\in E'} w_{a,b}x_{a,b}\\
\\
&\text{subject to}\\
&\forall~a \in E':~~~ \sum_{b \in E'} x_{ab} = 1 \\
&\forall~a \in E':~~~ \sum_{b \in E'} x_{ba} = 1 \\
&\forall~a,b \in E, a \neq b:~~~ u_a -u_b+1\leq (|E'|-1)(1-x_{ab}) \\
&\forall~a \in E: ~2\leq u_a \leq |E'|\\
&u\in \mathbb{N_{>0}}\\
&x\in \{0,1\}
\end{align}

with:

\begin{align}
& E: \text{Set of selected epitopes}\\
& \epsilon :\text{Dunmmy epitope} \\
& E' : \text{Epitope set with dummy epitope ($E' =E \cup \{\epsilon\})$}\\
& w_{ab} : \text{Edge weight (i.e negative Cleavage likelihood) between epitope $a$ and $b$} \\
& x_{ab} : \text{binary decision variable if edge between epitope $a$ and $b$ is selected} \\
& u_a : \text{Position of epitope $a$ in string-of-beads ($1\leq u_a\leq |E'|$)}
\end{align}

Task 4.1: Implement the MTZ-TSP formulation to solve the epitope assembly problem
---

Use what you have learned to formulate and solve the epitope assembly problem.

The following code can be used to predict cleavage likelihoods and to generate the fully-connected acyclic graph:

In [ ]:
import itertools as itr
from epytope.Core import Peptide
from epytope.CleavagePrediction import CleavageSitePredictorFactory

pep_tmp = <add here the list of selected neoepitopes as string>
pep_tmp.append("Dummy")
edge_matrix = {}
fragments = {}
pred = CleavageSitePredictorFactory("proteasmm_c")

for start, stop in itr.combinations(pep_tmp, 2):
    if start == "Dummy" or stop == "Dummy":
        edge_matrix[(str(start), str(stop))] = 0
        edge_matrix[(str(stop), str(start))] = 0
    else:
        start_str = str(start)
        stop_str = str(stop)
        frag = Peptide(start_str+stop_str)
        garf = Peptide(stop_str+start_str)

        fragments[frag] = (start_str, stop_str)
        fragments[garf] = (stop_str, start_str)

cleave_pred = pred.predict(fragments.keys())

for i in set(cleave_pred.index.get_level_values(0)):
    fragment = "".join(cleave_pred.loc[i]["Seq"])
    start, stop = fragments[fragment]
    cleav_pos = len(str(start)) - 1
    edge_matrix[(start, stop)] = -1.0 * (cleave_pred.loc[(i, len(str(start)) - 1), pred.name])
edge_matrix